In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from model import LSTM

In [2]:
print(LSTM)

<class 'model.LSTM'>


In [2]:



import time
tiempo_inicio = time.time()
tiempo_fin = time.time()
tiempo_ejecucion = (tiempo_fin - tiempo_inicio) * 60

def mes(row):
    if row['mes'] < 10:
        return '0' + str(row['mes'])
    else:
        return str(row['mes'])

###---------------------------------------------------------
df = pd.read_pickle('./Data/lstm_mensual_2.pkl')
df['mes'] = df.apply(lambda row: mes(row), axis=1)
df['Fecha_Inicio'] = df['year'].astype(str) + '-' + df['mes'].astype(str) + '-01'

df.Fecha_Inicio = pd.to_datetime(df.Fecha_Inicio)
df_exito = df[(df.Razon_Social_Cliente == 'Almacenes Exito Sa')& (df.Fecha_Inicio < '2023-11-01')].groupby(['Fecha_Inicio'],as_index=False).COP.sum()
df_exito = df_exito.set_index('Fecha_Inicio')
df_exito = df_exito.sort_index(ascending=False)





In [3]:
df_exito

,COP
Fecha_Inicio,
2023-10-01,1.718048e+09
2023-09-01,1.776914e+09
2023-08-01,1.626946e+09
2023-07-01,1.987764e+09
2023-06-01,1.820727e+09
...,...
2018-05-01,2.541605e+09
2018-04-01,1.881336e+09
2018-03-01,2.486371e+09


In [4]:
def create_sequences(data, seq_length, num_labels):
    sequences, labels = [], []
    for i in range(len(data) - seq_length - num_labels + 1):
        seq = np.array(data[i:i + seq_length])
        label = np.array(data[i + seq_length:(i + seq_length) + num_labels])
        sequences.append(seq)
        labels.append(label)
    return torch.tensor(sequences), torch.tensor(labels)
sequences, labels = create_sequences(df_exito, seq_length=5, num_labels=1)
print(sequences.shape)
print(labels.shape)


torch.Size([65, 5, 1])
torch.Size([65, 1, 1])


C:\Users\mario\AppData\Local\Temp\ipykernel_12080\4028785214.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.tensor(sequences), torch.tensor(labels)


In [5]:
labels

tensor([[[1.6165e+09]],

        [[1.4721e+09]],

        [[1.6540e+09]],

        [[1.5681e+09]],

        [[1.7249e+09]],

        [[2.2489e+09]],

        [[1.5393e+09]],

        [[1.8544e+09]],

        [[1.6759e+09]],

        [[1.7008e+09]],

        [[2.0970e+09]],

        [[1.8522e+09]],

        [[1.9160e+09]],

        [[1.8361e+09]],

        [[2.0682e+09]],

        [[1.8263e+09]],

        [[1.9797e+09]],

        [[2.7017e+09]],

        [[1.7473e+09]],

        [[1.9959e+09]],

        [[1.5206e+09]],

        [[1.5670e+09]],

        [[1.8764e+09]],

        [[1.5186e+09]],

        [[1.4093e+09]],

        [[1.2366e+09]],

        [[1.7932e+09]],

        [[1.6227e+09]],

        [[1.4827e+09]],

        [[2.2869e+09]],

        [[1.6814e+09]],

        [[2.0036e+09]],

        [[1.6714e+09]],

        [[1.4248e+09]],

        [[1.7119e+09]],

        [[1.6661e+09]],

        [[1.9559e+09]],

        [[1.4811e+09]],

        [[2.7138e+09]],

        [[1.9235e+09]],



In [55]:
sequences

tensor([[[1.7180e+09],
         [1.7769e+09],
         [1.6269e+09],
         [1.9878e+09],
         [1.8207e+09]],

        [[1.7769e+09],
         [1.6269e+09],
         [1.9878e+09],
         [1.8207e+09],
         [1.6165e+09]],

        [[1.6269e+09],
         [1.9878e+09],
         [1.8207e+09],
         [1.6165e+09],
         [1.4721e+09]],

        [[1.9878e+09],
         [1.8207e+09],
         [1.6165e+09],
         [1.4721e+09],
         [1.6540e+09]],

        [[1.8207e+09],
         [1.6165e+09],
         [1.4721e+09],
         [1.6540e+09],
         [1.5681e+09]],

        [[1.6165e+09],
         [1.4721e+09],
         [1.6540e+09],
         [1.5681e+09],
         [1.7249e+09]],

        [[1.4721e+09],
         [1.6540e+09],
         [1.5681e+09],
         [1.7249e+09],
         [2.2489e+09]],

        [[1.6540e+09],
         [1.5681e+09],
         [1.7249e+09],
         [2.2489e+09],
         [1.5393e+09]],

        [[1.5681e+09],
         [1.7249e+09],
         [2.2489e+

In [6]:

# splitting data to test and train
train_size = int(len(sequences) * 0.8)
test_size = len(sequences) - train_size
train_sequences, train_labels = sequences[:train_size], labels[:train_size]
test_sequences, test_labels = sequences[train_size:], labels[train_size:]
print(train_sequences.shape)
print(train_labels.shape)
print(test_sequences.shape)
print(test_labels.shape)

# create a dataset, for implementing the lstm algorithm
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, index):
        return self.sequences[index], self.labels[index]
    
train_dataset = Dataset(train_sequences, train_labels)
test_dataset = Dataset(test_sequences, test_labels)
# upload to dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

torch.Size([52, 5, 1])
torch.Size([52, 1, 1])
torch.Size([13, 5, 1])
torch.Size([13, 1, 1])


In [9]:
# algoritm for lstm for 3 features that will be the inputs and 2 features that will be the outputs
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1, num_layers=3):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        
        
        
    def forward(self, input_seq):
        self.hidden_cell = (torch.zeros(self.num_layers,input_seq.size(0),self.hidden_layer_size),
                            torch.zeros(self.num_layers,input_seq.size(0),self.hidden_layer_size))
        # Asegúrate de que la dimensión de entrada sea correcta
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), -1, 1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [14]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 500
for i in range(epochs):
    for seq, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(seq.float().reshape(-1,1))
        single_loss = loss_function(y_pred, labels.float())
        single_loss.backward()
        optimizer.step()
    if i%2 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

C:\Users\mario\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([52, 1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:   1 loss: 3661960035069591552.00000000
epoch:   3 loss: 3661960035069591552.00000000
epoch:   5 loss: 3661960035069591552.00000000
epoch:   7 loss: 3661959760191684608.00000000
epoch:   9 loss: 3661959760191684608.00000000
epoch:  11 loss: 3661960309947498496.00000000
epoch:  13 loss: 3661960035069591552.00000000
epoch:  15 loss: 3661959760191684608.00000000
epoch:  17 loss: 3661959760191684608.00000000
epoch:  19 loss: 3661960035069591552.00000000
epoch:  21 loss: 3661960035069591552.00000000
epoch:  23 loss: 3661959760191684608.00000000
epoch:  25 loss: 3661960035069591552.00000000
epoch:  27 loss: 3661960035069591552.00000000
epoch:  29 loss: 3661960035069591552.00000000
epoch:  31 loss: 3661960035069591552.00000000
epoch:  33 loss: 3661960035069591552.00000000
epoch:  35 loss: 3661959760191684608.00000000
epoch:  37 loss: 3661960035069591552.00000000
epoch:  39 loss: 3661959760191684608.00000000
epoch:  41 loss: 3661959760191684608.00000000
epoch:  43 loss: 36619600350695915

In [18]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))

        h_out = h_out.view(-1, self.hidden_size)

        out = self.fc(h_out)

        return out

In [ ]:
num_epochs = 500
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()

    # obtain the loss function
    loss = criterion(outputs, trainY)

    loss.backward()

    optimizer.step()
    data_predict = outputs.data.numpy()
    dataY_plot = trainY.data.numpy()
    rmse =  np.sqrt(mean_squared_error(dataY_plot,data_predict))
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f, rmse: %0.f" % (epoch, loss.item(),rmse))

In [4]:
# Train test split ------------------------ Window view def sliding_windows(data, seq_length):
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(df_exito)

seq_length = 3
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.80)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))
testY.shape,trainY.shape,testX.shape,trainX.shape

(torch.Size([14, 1]),
 torch.Size([53, 1]),
 torch.Size([14, 3, 1]),
 torch.Size([53, 3, 1]))

In [69]:
trainY

tensor([[0.3174],
        [0.2468],
        [0.1605],
        [0.0995],
        [0.1764],
        [0.1401],
        [0.2064],
        [0.4278],
        [0.1279],
        [0.2611],
        [0.1856],
        [0.1962],
        [0.3636],
        [0.2601],
        [0.2871],
        [0.2533],
        [0.3514],
        [0.2492],
        [0.3140],
        [0.6192],
        [0.2158],
        [0.3209],
        [0.1200],
        [0.1396],
        [0.2704],
        [0.1192],
        [0.0730],
        [0.0000],
        [0.2352],
        [0.1631],
        [0.1040],
        [0.4438],
        [0.1880],
        [0.3241],
        [0.1837],
        [0.0795],
        [0.2009],
        [0.1815],
        [0.3040],
        [0.1033],
        [0.6243],
        [0.2903],
        [0.3686],
        [0.8182],
        [0.2151],
        [0.3058],
        [0.2943],
        [0.3169],
        [0.4000],
        [0.4388],
        [0.4645],
        [0.2824],
        [0.4954]])

In [65]:
training_data

array([[0.20344741],
       [0.22832548],
       [0.16494633],
       [0.3174342 ],
       [0.24684165],
       [0.16051058],
       [0.09951947],
       [0.1763594 ],
       [0.14006367],
       [0.20636404],
       [0.42779528],
       [0.12790886],
       [0.26106697],
       [0.18563728],
       [0.1961531 ],
       [0.36360709],
       [0.26014296],
       [0.28711494],
       [0.25332519],
       [0.35141444],
       [0.24917775],
       [0.31404204],
       [0.61917257],
       [0.21582119],
       [0.32087431],
       [0.119998  ],
       [0.13962141],
       [0.27035317],
       [0.11915673],
       [0.07298417],
       [0.        ],
       [0.23518747],
       [0.16313112],
       [0.10399878],
       [0.44384596],
       [0.18797634],
       [0.32413378],
       [0.18374765],
       [0.07953415],
       [0.2008586 ],
       [0.18148693],
       [0.30395526],
       [0.10330469],
       [0.62425264],
       [0.29025822],
       [0.36857542],
       [0.81822425],
       [0.215

In [5]:
import warnings
warnings.filterwarnings("ignore")
np.random.seed(40)
num_epochs = 500
learning_rate = 0.01

input_size = 1
hidden_size = 256
num_layers = 3
weight_decay = 0.01

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()  # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=weight_decay)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()

    # obtain the loss function
    loss = criterion(outputs, trainY)

    loss.backward()

    optimizer.step()
    data_predict = outputs.data.numpy()
    dataY_plot = trainY.data.numpy()

    data_predict = sc.inverse_transform(data_predict.reshape(-1,1))
    dataY_plot = sc.inverse_transform(dataY_plot.reshape(-1,1))
    print(data_predict.shape)
    print(dataY_plot.shape)
    #rmse = np.sqrt(mean_squared_error(dataY_plot, data_predict))
    #mse = mean_squared_error(dataY_plot, data_predict)
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f, rmse: %0.f" % (epoch, mse, rmse))

IndexError: too many indices for tensor of dimension 2

In [16]:
sc.inverse_transform(data_predict.reshape(-1,1))

array([[1.3272138e+09]], dtype=float32)

In [22]:
warnings.filterwarnings("ignore")
np.random.seed(40)
num_epochs = 500
learning_rate = 0.01

input_size = 1
hidden_size = 256
num_layers = 3
weight_decay = 0.01

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
lstm(trainX).shape

torch.Size([159, 1])

In [10]:
53 * 3

159